# Load Library

In [1]:
suppressMessages(library(data.table))
suppressMessages(library(tidyverse))  
suppressMessages(library(lightgbm)) 

# Read Data

In [2]:
train_preprocess = fread("../preprocess/train_preprocess.csv", na.strings=c("NA","NaN", ""),encoding="UTF-8")

# Data for Modeling

In [3]:
# Binary 학습을 위한 is_survive, 전체 결제금액 학습을 위한 total_amount_spent 파생변수 생성 
train_preprocess = train_preprocess  %>%
    mutate(is_survive = ifelse(survival_time==64,1,0),
          total_amount_spent = survival_time * amount_spent)

In [5]:
# split


# --------Survival_time----------
x_train = train_preprocess  %>% 
            select(-c("acc_id","survival_time","amount_spent","is_survive","total_amount_spent"))

y_train = train_preprocess  %>% 
            select(is_survive)


# --------Total_Amount_Spent----------
x_train2 = train_preprocess  %>% 
            filter(total_amount_spent!=0)  %>% 
            select(-c("acc_id","survival_time","amount_spent","is_survive","total_amount_spent"))

y_train2 = train_preprocess  %>% 
            filter(total_amount_spent!=0)  %>% 
            select(total_amount_spent)

In [6]:
# --------Survival_time----------
x_train<- model.matrix(~.-1, data = x_train)
y_train<- model.matrix(~.-1, data = y_train) 


# --------Total_Amount_Spent----------
x_train2<- model.matrix(~.-1, data = x_train2)
y_train2<- model.matrix(~.-1, data = y_train2) 

# Light GBM

In [ ]:
# 이탈 / 비이탈
lgb.train = lgb.Dataset(data=x_train, label=y_train)

# 전체 결제 금액
lgb.train2 = lgb.Dataset(data=x_train2, label=y_train2)

In [ ]:
lgb.grid = list(objective = "binary",
                metric = "auc",
                learning_rate = 0.03,
                alpha = 0.5,
                lambda = 0.5,
                max_depth = 7)

In [ ]:
lgb.grid2 = list(objective = "regression",
                 metric = "rmse",
                 learning_rate = 0.01,
                 alpha = 0.5,
                 lambda = 0.5,
                 max_depth = 7)

In [ ]:
set.seed(828)

# 이탈 / 비이탈
lgb.fit = lgb.train(params = lgb.grid, data = lgb.train, nrounds =1879, eval_freq=100)


# 전체 결제 금액
lgb.fit2 = lgb.train(params = lgb.grid2, data = lgb.train2, nrounds=678, eval_freq=100)

In [ ]:
lgb.save(lgb.fit, "../model/final_1.model")
lgb.save(lgb.fit2, "../model/final_2.model")